In [1]:
import re
from pathlib import Path
import pandas as pd
import time
from pathlib import Path
from PIL import Image
import numpy as np
import cv2
from tqdm import tqdm

from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.cluster import MiniBatchKMeans
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from  sklearn.utils import parallel_backend

from sklearn.exceptions import ConvergenceWarning
import warnings
warnings.filterwarnings("ignore", category = ConvergenceWarning)

# Load Images 

In [2]:

def get_images_and_labels(path, file_ending= "jpg"):
    paths = []
    labels = []
    for directory in Path(path).glob('*'):
        for p in directory.glob("*." + file_ending):
            paths.append(p)
            labels.append(directory.name)

    return paths, labels



fruit_image_paths, fruit_image_labels = get_images_and_labels("data/FIDS30/")

traffic_sign_image_paths_train, traffic_sign_image_labels_train = get_images_and_labels("data/TrafficSigns/train", "ppm")
traffic_sign_image_paths_test, traffic_sign_image_labels_test = get_images_and_labels("data/TrafficSigns/test","ppm")


# Prepare Features 

In [3]:


def color_histogram_features(images):
   
    # Next, we extract a few more features using OpenCV
    dataOpenCV_1D = []
    dataOpenCV_2D = []
    dataOpenCV_3D = []

    # use our own simple function to flatten the 2D arrays
    flatten = lambda l: [item for sublist in l for item in sublist]

    for filepath in tqdm(images):

        # the easiest way would to do the following:
        # imageOpenCV = cv2.imread(imagePath + fileName)

        # However, we have the same issue as before, and it is more difficult in OpenCV to convert to an RGB image
        # Thus we do this using PIL, and then convert to OpenCV ....
        imagePIL = Image.open(filepath)
        imagePIL = imagePIL.convert('RGB')
        imageOpenCV = np.array(imagePIL)
        # Convert RGB to BGR
        imageOpenCV = imageOpenCV[:, :, ::-1].copy()

        # Now we split the image in the three channels, B / G / R
        chans = cv2.split(imageOpenCV)
        colors = ("b", "g", "r")

        # First we do also features per channel, but this time, we aggregate them into a smaller number of bins
        # I.e. we do not have 256 values per channel, but less
        featuresOpenCV_1D = []
        bins_1D = 64
        for (chan, color) in zip(chans, colors):  # we compute the histogram over each channel
            histOpenCV = cv2.calcHist([chan], [0], None, [bins_1D], [0, 256])
            featuresOpenCV_1D.extend(histOpenCV)
        featureVectorOpenCV_1D = flatten(featuresOpenCV_1D)  # and append this to our feature vector

        dataOpenCV_1D.append(featureVectorOpenCV_1D)  # now we append the feature vector to the dataset so far

        if (len(featureVectorOpenCV_1D) != bins_1D * 3):  # sanity check, in case we had a wrong number of channels...
            print("Unexpected length of feature vector: " + str(len(featureVectorOpenCV_1D)) + " in file: " + filepath)

        # Next - features that look at two channels at the same time
        # E.g. we look at when green and blue have both "high values"
        # We reduce the size of bins further, to not have a too long feature vector
        featuresOpenCV_2D = []
        bins2D = 16
        # look at all combinations of channels (R & B, R & G, B & G)
        featuresOpenCV_2D.extend(cv2.calcHist([chans[1], chans[0]], [0, 1], None, [bins2D, bins2D], [0, 256, 0, 256]))
        featuresOpenCV_2D.extend(cv2.calcHist([chans[1], chans[2]], [0, 1], None, [bins2D, bins2D], [0, 256, 0, 256]))
        featuresOpenCV_2D.extend(cv2.calcHist([chans[0], chans[2]], [0, 1], None, [bins2D, bins2D], [0, 256, 0, 256]))
        # and add that to our dataset
        featureVectorOpenCV_2D = flatten(featuresOpenCV_2D)
        dataOpenCV_2D.append(featureVectorOpenCV_2D)

        # finally, we look at all three channels at the same time.
        # We further reduce our bin size, because otherwise, this would become very large...
        featuresOpenCV_3D = cv2.calcHist([imageOpenCV], [0, 1, 2], None, [8, 8, 8], [0, 256, 0, 256, 0, 256])
        # append to our dataset
        featureVectorOpenCV_3D = featuresOpenCV_3D.flatten()
        dataOpenCV_3D.append(featureVectorOpenCV_3D)

    return dataOpenCV_1D, dataOpenCV_2D, dataOpenCV_3D


In [4]:

def bovw(images,kmeans):
    
    # let's use opencv to extract the features
    sift = cv2.xfeatures2d.SIFT_create()
    features_all = []
    feature_counts = []

    # for every path extract the 128 sift descriptors
    for path in tqdm(images):
        img = cv2.imread(str(path), 0)
        _, features = sift.detectAndCompute(img, None)
        if features is not None:
            feature_counts.append(features.shape[0])
            features_all.append(features)
        else:
            feature_counts.append(0)

    features_all = np.vstack(features_all)
    display("Do KMeans clustering")
    n_clusters = 100
    # assign a cluster/word to every feature that SIFT found
    if kmeans is None:
        kmeans = MiniBatchKMeans(n_clusters = n_clusters)
        kmeans.fit(features_all)
        clusters = kmeans.predict(features_all)
    else:
        clusters = kmeans.predict(features_all)

    histogram = np.zeros((len(images), n_clusters))
    i = 0
    for image_idx, count in enumerate(feature_counts):
        for c in range(i, i+count):
            histogram[image_idx][clusters[c]] += 1
        i += count

    histogram = StandardScaler().fit_transform(histogram)

    return histogram, kmeans





# Classifiers 

We are using three different classifiers. 

In [5]:
def train_test(X_train, X_test, y_train, y_test, params ) :
   
    results = []

    for model_name, param in params.items():
        display("Hyperparameter search for model :" + model_name)
        clf = GridSearchCV(param['model'], param['parameters'], scoring='accuracy', cv=3, n_jobs=5)
        clf.fit(X_train, y_train)
        
        start_time = time.time()
        y_pred = clf.predict(X_test)
        predict_duration = time.time() - start_time
        accuracy = accuracy_score(y_test, y_pred)
        
        results.append({
            'model': model_name,
            'params': '_'.join([f'{k}={v}' for k, v in clf.best_params_.items()]),
            'y_pred': y_pred,
            'y_true': y_test,
            'train_duration': clf.refit_time_,
            'predict_duration': predict_duration,
            'accuracy': accuracy
        })

    return results

In [18]:
params = {
    'RandomForrest': {
        'model': RandomForestClassifier(n_jobs = 2,random_state=1234),
        'parameters': {
            'n_estimators': [10, 25, 50, 100, 200],
            'max_depth': [4, 8, 15, 20]
        }
    },
    'MLP': {
        'model': MLPClassifier( max_iter = 500, random_state=1234),
        'parameters': {
            'hidden_layer_sizes': [
                (64,),
                (128,),
                (128, 64),
            ],
            'activation': ['tanh'],
            'alpha': [0.01, 0.1],
            'learning_rate' : ['constant'],
            'learning_rate_init' : [ 0.01, 0.001]
        }
    },
    'k-NN': {
        'model': KNeighborsClassifier(),
        'parameters': {'n_neighbors': [3, 5, 10, 30, 50]}
    }
}

# Color Histogram Classification Fruits

In [7]:
import time 

display("Calculate Histograms for Fruits!")
feature_extraction_start_time = time.time()
opencv_1D, opencv_2D, opencv_3D = color_histogram_features(fruit_image_paths)
duration =  time.time() - feature_extraction_start_time
display("Feature extraction took: " + str(duration) + " seconds")

'Calculate Histograms for Fruits!'

100%|██████████| 971/971 [00:55<00:00, 17.56it/s]


'Feature extraction took: 55.29846501350403 seconds'

In [8]:

with parallel_backend('threading'):
    display('1D Train/test split')
    X_train, X_test, y_train, y_test = train_test_split(opencv_1D, fruit_image_labels, test_size=0.25, random_state=1234)
    display('Training on single channel histogram')
    results_1d = train_test(X_train, X_test, y_train, y_test,params)
    for r in results_1d:
        r['features'] = f'hist1D'

    display('2D Train/test split')
    X_train, X_test, y_train, y_test = train_test_split(opencv_2D, fruit_image_labels, test_size=0.25, random_state=1234)
    display('Training on 2D channel historgrams')
    results_2d = train_test(X_train, X_test, y_train, y_test,params)
    for r in results_2d:
        r['features'] = f'hist2D'

    display('3D Train/test split')
    X_train, X_test, y_train, y_test = train_test_split(opencv_3D, fruit_image_labels, test_size=0.25, random_state=1234)
    display('Training on 3D channel historgrams')
    results_3d = train_test(X_train, X_test, y_train, y_test,params)
    for r in results_3d:
        r['features'] = f'hist3D'

'1D Train/test split'

'Training on single channel histogram'

'Hyperparameter search for model :RandomForrest'

'Hyperparameter search for model :MLP'

'Hyperparameter search for model :k-NN'

...

'3D Train/test split'

'Training on 3D channel historgrams'

'Hyperparameter search for model :RandomForrest'

'Hyperparameter search for model :MLP'

'Hyperparameter search for model :k-NN'

In [9]:
fruits_hist_results_df = pd.DataFrame(results_1d + results_2d + results_3d) 

# BOVW Fruits

In [10]:
display("Calculate BOVW for Fruits!")

train_images, test_images, y_train, y_test = train_test_split(fruit_image_paths, fruit_image_labels, test_size=0.25, random_state=1234)

feature_extraction_start_time = time.time()
X_train, kmeans = bovw(train_images, None)
X_test, kmeans = bovw(test_images, kmeans=kmeans)
duration =  time.time() - feature_extraction_start_time

display("Feature extraction took: " + str(duration) + " seconds")

'Calculate BOVW for Fruits!'

100%|██████████| 728/728 [03:43<00:00,  3.25it/s]


'Do KMeans clustering'

100%|██████████| 243/243 [01:09<00:00,  3.49it/s]


'Do KMeans clustering'

'Feature extraction took: 374.25863695144653 seconds'

In [11]:
with parallel_backend('threading'):
    results = train_test(X_train, X_test, y_train, y_test, params )
    for r in results:
        r['features'] = f'bovw'
    fruits_bovw_results_df = pd.DataFrame(results)

'Hyperparameter search for model :RandomForrest'

'Hyperparameter search for model :MLP'

'Hyperparameter search for model :k-NN'

In [12]:
fruits_bovw_results_df

,model,params,y_pred,y_true,train_duration,predict_duration,accuracy,features
0,RandomForrest,max_depth=20_n_estimators=100,"[pineapples, apples, plums, raspberries, banan...","[bananas, mangos, cherries, blueberries, olive...",0.342589,0.106378,0.205761,bovw
1,MLP,activation=tanh_alpha=0.1_hidden_layer_sizes=(...,"[bananas, apricots, olives, plums, bananas, pe...","[bananas, mangos, cherries, blueberries, olive...",2.321604,0.000655,0.341564,bovw
2,k-NN,n_neighbors=10,"[bananas, bananas, apples, blueberries, banana...","[bananas, mangos, cherries, blueberries, olive...",0.003317,0.037328,0.181070,bovw


# Color Histogram Classification Traffic Signs


In [13]:
traffic_image_paths = traffic_sign_image_paths_train+traffic_sign_image_paths_test
display("Calculate Histograms for Traffic Signs!")
feature_extraction_start_time = time.time()
opencv_1D, opencv_2D, opencv_3D = color_histogram_features(traffic_image_paths)
duration =  time.time() - feature_extraction_start_time
display("Feature extraction took: " + str(duration) + " seconds")

'Calculate Histograms for Traffic Signs!'

100%|██████████| 6058/6058 [00:06<00:00, 923.19it/s] 


'Feature extraction took: 6.6029579639434814 seconds'

In [14]:
traffic_labels = traffic_sign_image_labels_train + traffic_sign_image_labels_test
with parallel_backend('threading'):
    display('1D Train/test split')
    X_train, X_test, y_train, y_test = train_test_split(opencv_1D, traffic_labels, test_size=0.25, random_state=1234)
    display('Training on single channel histogram')
    results_1d = train_test(X_train, X_test, y_train, y_test,params)
    for r in results_1d:
        r['features'] = f'hist1D'

    display('2D Train/test split')
    X_train, X_test, y_train, y_test = train_test_split(opencv_2D, traffic_labels, test_size=0.25, random_state=1234)
    display('Training on 2D channel historgrams')
    results_2d = train_test(X_train, X_test, y_train, y_test,params)
    for r in results_2d:
        r['features'] = f'hist2D'

    display('3D Train/test split')
    X_train, X_test, y_train, y_test = train_test_split(opencv_3D, traffic_labels, test_size=0.25, random_state=1234)
    display('Training on 3D channel historgrams')
    results_3d = train_test(X_train, X_test, y_train, y_test,params)
    for r in results_3d:
        r['features'] = f'hist3D'

'1D Train/test split'

'Training on single channel histogram'

'Hyperparameter search for model :RandomForrest'

'Hyperparameter search for model :MLP'

'Hyperparameter search for model :k-NN'

...

'3D Train/test split'

'Training on 3D channel historgrams'

'Hyperparameter search for model :RandomForrest'

'Hyperparameter search for model :MLP'

'Hyperparameter search for model :k-NN'

In [15]:
traffic_signs_hist_results_df = pd.DataFrame(results_1d + results_2d + results_3d) 

# BOVW Traffic Signs

In [16]:
display("Calculate BOVW for Traffic Signs!")

train_images, test_images, y_train, y_test = train_test_split(traffic_image_paths, traffic_labels, test_size=0.25, random_state=1234)

feature_extraction_start_time = time.time()
X_train, kmeans = bovw(train_images, None)
X_test, kmeans = bovw(test_images, kmeans=kmeans)
duration =  time.time() - feature_extraction_start_time

display("Feature extraction took: " + str(duration) + " seconds")

'Calculate BOVW for Traffic Signs!'

100%|██████████| 4543/4543 [00:04<00:00, 1010.90it/s]


'Do KMeans clustering'

100%|██████████| 1515/1515 [00:01<00:00, 1011.58it/s]


'Do KMeans clustering'

'Feature extraction took: 9.558998107910156 seconds'

In [19]:
with parallel_backend('threading'):
    results = train_test(X_train, X_test, y_train, y_test, params )
    for r in results:
        r['features'] = f'bovw'
    traffic_signs_bovw_results_df = pd.DataFrame(results)

'Hyperparameter search for model :RandomForrest'

'Hyperparameter search for model :MLP'

'Hyperparameter search for model :k-NN'

# Results Helper

In [20]:
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels

def plotConfusionMatrix(y_true, y_pred, classes, name=None, normalize=True,
                          cmap=plt.cm.Blues):

    cm = confusion_matrix(y_true, y_pred)
    classes = np.array(list(classes))[unique_labels(y_true, y_pred)]
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    fig, ax = plt.subplots(figsize=(10,8))
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)

    ax.set(xticks=np.arange(cm.shape[1]), yticks=np.arange(cm.shape[0]),
           xticklabels=classes, yticklabels=classes,
           title='Confusion Matrix',
           ylabel='True label',
           xlabel='Predicted label')

    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")

    fmt = '.2f' if normalize else 'd'
    size = 6 if normalize else 9
    thresh = 0.5 if normalize else cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center", size=size,
                    color="white" if cm[i, j] > thresh else "black")

    fig.tight_layout()

    if name is not None:
        plt.savefig('./figures/' + name + '_cm.png', dpi=150)
        plt.close()

In [29]:
 # print results
def print_and_save_results(results, labels):
    for i, r in results.iterrows():
        classes = list(set(labels))
        y_true = np.array([classes.index(y) for y in r['y_true']])
        y_pred = np.array([classes.index(y) for y in r['y_pred']])
        accuracy = sum(y_true == y_pred) / y_true.shape[0]
        dataset_name = "fruits"
        plotConfusionMatrix(
            y_true=y_true,
            y_pred=y_pred,
            classes=classes,
            name=f'{r["model"]} {dataset_name} {r["features"]} {r["params"]}'
        )


        with open(f'scores/{r["model"]} {dataset_name} {r["features"]} {r["params"]}.txt', 'w') as f:
            f.write('model,features,parameters,accuracy\n')
            f.write(f'{r["model"]},{r["features"]},"{r["params"]}",{accuracy}')
            

# Results Fruits

In [30]:
print_and_save_results(fruits_hist_results_df,fruit_image_labels)


In [31]:
pd.set_option('display.max_colwidth', -1)
fruits_hist_results_df[["model","params", "train_duration", "predict_duration","accuracy", "features"]]


,model,params,train_duration,predict_duration,accuracy,features
0,RandomForrest,max_depth=15_n_estimators=200,0.899194,0.112549,0.271605,hist1D
1,MLP,"activation=relu_alpha=0.1_hidden_layer_sizes=(256,)_learning_rate=constant_learning_rate_init=0.001",4.715840,0.005972,0.267490,hist1D
2,k-NN,n_neighbors=10,0.020377,0.042981,0.123457,hist1D
3,RandomForrest,max_depth=20_n_estimators=200,1.039364,0.122725,0.460905,hist2D
4,MLP,"activation=relu_alpha=0.1_hidden_layer_sizes=(256,)_learning_rate=constant_learning_rate_init=0.001",4.169756,0.020465,0.440329,hist2D
5,k-NN,n_neighbors=10,0.073017,0.148042,0.172840,hist2D
6,RandomForrest,max_depth=20_n_estimators=200,0.686426,0.105014,0.489712,hist3D
7,MLP,"activation=tanh_alpha=1_hidden_layer_sizes=(256, 128)_learning_rate=adaptive_learning_rate_init=0.001",2.427944,0.002454,0.411523,hist3D
8,k-NN,n_neighbors=10,0.014134,0.082800,0.213992,hist3D


In [32]:
print_and_save_results(fruits_bovw_results_df,fruit_image_labels)


In [33]:
pd.set_option('display.max_colwidth', -1)
fruits_bovw_results_df[["model","params", "train_duration", "predict_duration","accuracy", "features"]]


,model,params,train_duration,predict_duration,accuracy,features
0,RandomForrest,max_depth=20_n_estimators=100,0.342589,0.106378,0.205761,bovw
1,MLP,"activation=tanh_alpha=0.1_hidden_layer_sizes=(64,)_learning_rate=constant_learning_rate_init=0.001",2.321604,0.000655,0.341564,bovw
2,k-NN,n_neighbors=10,0.003317,0.037328,0.181070,bovw


# Results Traffic Signs

In [35]:
print_and_save_results(traffic_signs_hist_results_df, traffic_sign_image_labels_train+traffic_sign_image_labels_test)


In [36]:
pd.set_option('display.max_colwidth', -1)
traffic_signs_hist_results_df[["model","params", "train_duration", "predict_duration","accuracy", "features"]]


,model,params,train_duration,predict_duration,accuracy,features
0,RandomForrest,max_depth=20_n_estimators=200,2.295731,0.137406,0.893729,hist1D
1,MLP,"activation=tanh_alpha=1_hidden_layer_sizes=(256,)_learning_rate=adaptive_learning_rate_init=0.001",3.016250,0.029408,0.919472,hist1D
2,k-NN,n_neighbors=3,0.113536,0.794551,0.925413,hist1D
3,RandomForrest,max_depth=20_n_estimators=200,2.252907,0.193570,0.932673,hist2D
4,MLP,"activation=tanh_alpha=0.01_hidden_layer_sizes=(256,)_learning_rate=constant_learning_rate_init=0.001",7.798414,0.101383,0.955116,hist2D
5,k-NN,n_neighbors=3,0.450569,1.795464,0.933993,hist2D
6,RandomForrest,max_depth=20_n_estimators=200,1.201391,0.106719,0.910231,hist3D
7,MLP,"activation=tanh_alpha=1_hidden_layer_sizes=(256,)_learning_rate=adaptive_learning_rate_init=0.001",7.576664,0.009260,0.948515,hist3D
8,k-NN,n_neighbors=3,0.146073,1.210200,0.920132,hist3D


In [37]:
print_and_save_results(traffic_signs_bovw_results_df,traffic_sign_image_labels_train+traffic_sign_image_labels_test)


In [38]:
pd.set_option('display.max_colwidth', -1)
traffic_signs_bovw_results_df[["model","params", "train_duration", "predict_duration","accuracy", "features"]]


,model,params,train_duration,predict_duration,accuracy,features
0,RandomForrest,max_depth=20_n_estimators=200,0.569251,0.103626,0.877228,bovw
1,MLP,"activation=tanh_alpha=0.1_hidden_layer_sizes=(64,)_learning_rate=constant_learning_rate_init=0.001",3.489731,0.001043,0.882508,bovw
2,k-NN,n_neighbors=5,0.037804,1.285195,0.835644,bovw
